### Chromadb and embedded version with lastest version of packages [Semantic Search Version ]

In [1]:
# %env OPENAI_API_KEY=Your API Key

### Uncommand the below pip command only if the packages version is old even you install new one, once u uninstalled rerun the pip install

##### you can check the packages version using the last cell

In [2]:
# !pip uninstall -y pandas openai langchain langchain-openai langchain-huggingface langchain-core langchain-text-splitters langchain-chroma chromadb sentence-transformers pydantic

### Uncommand the below pip command only if you didn't installed the packages already

In [3]:
# !pip install -U -q pandas openai langchain langchain-openai langchain-huggingface langchain-core langchain-text-splitters langchain-chroma chromadb sentence-transformers pydantic

In [4]:
import os
import json
import textwrap
import pandas as pd
from pydantic import BaseModel
from typing import Optional

from openai import OpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [5]:
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(
    api_key=api_key,
    base_url="https://openai.vocareum.com/v1"
)

# Helper LLM call
def gpt(prompt, temp=0.0, print_want=True):
    print("LLM Started....")
    resp = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=temp,
    )

    output = resp.choices[0].message.content.strip()
    if print_want:
        print(output)
    return output

In [6]:
# 2. CSV Generation (same as v1)

headers = "Neighborhood,Price,Bedroom,Bathroom,House_size,Description"

csv_prompt = f"""
Generate 10 synthetic house listings in strict CSV format.
Headers: {headers}

Rules:
1. Output must contain ONLY the header + 10 rows.
2. Do NOT use commas in numeric fields.
3. Description must not contain commas.
4. All fields must be wrapped in double quotes.
"""

csv_text = gpt(csv_prompt)

LLM Started....
Neighborhood,Price,Bedroom,Bathroom,House_size,Description
"Midtown",250000,3,2,2000,"Beautiful single-family home with modern finishes"
"Downtown",350000,4,3,2500,"Spacious townhouse with rooftop deck"
"Suburbia",200000,2,1,1500,"Cozy starter home with fenced backyard"
"Waterfront",500000,5,4,3000,"Luxurious waterfront property with private dock"
"Mountainside",400000,4,3,2800,"Custom-built mountain retreat with stunning views"
"Beachfront",600000,6,5,3500,"Elegant beachfront estate with pool and guest house"
"City Center",300000,3,2,1800,"Convenient city living in a newly renovated townhome"
"Family Friendly",275000,3,2,1900,"Spacious family home in a quiet neighborhood"
"Historic District",450000,4,3,2600,"Charming historic home with original details"
"Country Living",325000,3,2,2100,"Peaceful country home on acres of land"


In [7]:
with open("homes.csv", "w", encoding="utf-8") as f:
    f.write(csv_text)

df = pd.read_csv("homes.csv")
print(df.head())

   Neighborhood   Price  Bedroom  Bathroom  House_size  \
0       Midtown  250000        3         2        2000   
1      Downtown  350000        4         3        2500   
2      Suburbia  200000        2         1        1500   
3    Waterfront  500000        5         4        3000   
4  Mountainside  400000        4         3        2800   

                                         Description  
0  Beautiful single-family home with modern finishes  
1               Spacious townhouse with rooftop deck  
2             Cozy starter home with fenced backyard  
3    Luxurious waterfront property with private dock  
4  Custom-built mountain retreat with stunning views  


In [8]:
def row_to_text(row):
    return (
        f"{row['Neighborhood']} "
        f"{row['Price']} "
        f"{row['Bedroom']} Bedroom "
        f"{row['Bathroom']} Bathroom "
        f"{row['House_size']} "
        f"{row['Description']}"
    )

documents = []
for i, row in df.iterrows():
    text = row_to_text(row)
    documents.append(Document(
        page_content=text,
        metadata=row.to_dict()
    ))

In [9]:
try:
    embeddings = OpenAIEmbeddings(
        model="text-embedding-3-small",
        api_key=api_key,
        openai_api_base="https://openai.vocareum.com/v1"
    )
    print("Using OpenAI embeddings")
except:
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    print("Using HF embeddings")

Using OpenAI embeddings


In [10]:
db = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    collection_name="homes_vdb"
)

print("Vector DB created!")

Vector DB created!


In [11]:
class Prefs(BaseModel):
    budget: Optional[int] = None
    bedroom: Optional[int] = None
    bathroom: Optional[int] = None
    min_size: Optional[int] = None
    preferances: Optional[str] = None

example = "3 bedroom and 2 bathroom under $500000 with minimum 100 sqft with a mountain view"
user_text = input(f"Enter your preferences (ex: {example}) : ")
if not user_text.strip():
    user_text = example

extract_prompt = f"""
Extract JSON from the following text:

\"\"\"{user_text}\"\"\"

Return JSON in this format:
{{
 "budget": int|null,
 "bedroom": int|null,
 "bathroom": int|null,
 "min_size": int|null,
 "preferances": string|null
}}
"""

prefs_json = gpt(extract_prompt)
prefs = Prefs(**json.loads(prefs_json))

Enter your preferences (ex: 3 bedroom and 2 bathroom under $500000 with minimum 100 sqft with a mountain view) :  3 bedroom and 2 bathroom under $500000 with a mountain view


LLM Started....
{
 "budget": 500000,
 "bedroom": 3,
 "bathroom": 2,
 "min_size": null,
 "preferances": "mountain view"
}


In [12]:
# Ask for missing fields
missing = [f for f, v in prefs.model_dump().items() if v is None and f != "preferances"]
if missing:
    print("Missing fields:", missing)
    for m in missing:
        val = input(f"Enter {m} (or leave blank): ")
        if val.strip():
            try:
                setattr(prefs, m, int(val))
            except:
                pass

print("\nFinal Prefs:", prefs.model_dump())

Missing fields: ['min_size']


Enter min_size (or leave blank):  500



Final Prefs: {'budget': 500000, 'bedroom': 3, 'bathroom': 2, 'min_size': 500, 'preferances': 'mountain view'}


In [13]:
# ----------------------------------------------------------------
# 7. Query Vector DB (semantic similarity)
# ----------------------------------------------------------------
query_text = " ".join([
    prefs.preferances or "",
    f"{prefs.bedroom} bedroom" if prefs.bedroom else "",
    f"{prefs.bathroom} bathroom" if prefs.bathroom else "",
    f"{prefs.budget} budget" if prefs.budget else "",
    f"{prefs.min_size} sqft" if prefs.min_size else "",
])

results = db.similarity_search(query_text, k=10)

In [14]:
filtered = []
for r in results:
    m = r.metadata
    price = int(str(m["Price"]).replace("$", "").replace(",", ""))
    size = int(str(m["House_size"]).replace("sqft", "").strip())

    ok = True
    if prefs.budget and price > prefs.budget: ok = False
    if prefs.bedroom and m["Bedroom"] < prefs.bedroom: ok = False
    if prefs.bathroom and m["Bathroom"] < prefs.bathroom: ok = False
    if prefs.min_size and size < prefs.min_size: ok = False

    if ok:
        filtered.append(r)

In [15]:
print("\n---- TOP MATCHES ----")
for i, r in enumerate(filtered[:3], 1):
    print(f"\n🏠 Home {i}")
    for k, v in r.metadata.items():
        print(f"{k}: {v}")

if not filtered:
    print("No matches found!")


---- TOP MATCHES ----

🏠 Home 1
Bedroom: 4
House_size: 2800
Neighborhood: Mountainside
Bathroom: 3
Price: 400000
Description: Custom-built mountain retreat with stunning views

🏠 Home 2
Bedroom: 3
House_size: 1800
Description: Convenient city living in a newly renovated townhome
Neighborhood: City Center
Price: 300000
Bathroom: 2

🏠 Home 3
House_size: 2500
Neighborhood: Downtown
Bathroom: 3
Price: 350000
Description: Spacious townhouse with rooftop deck
Bedroom: 4


In [16]:
if filtered:
    best = filtered[0].metadata
    rec_prompt = f"""
Create a short personalized home recommendation using these preferences and the matched home.
Add emojis.

User Prefs:
{json.dumps(prefs.model_dump(), indent=5)}

Matched Home:
{json.dumps(best, indent=5)}
"""
    recommendation = gpt(rec_prompt, temp=0.7)
    print("\n🎯 Personalized Recommendation:\n")
    print(textwrap.fill(recommendation, width=100))


LLM Started....
I found the perfect home for you! 🏡 This custom-built mountain retreat in the Mountainside neighborhood offers 4 bedrooms, 3 bathrooms, and stunning views. With a spacious house size of 2800 sqft, this home is listed below your budget at $400,000. Don't miss out on this dreamy mountain view property! 🌄 #HomeSweetHome

🎯 Personalized Recommendation:

I found the perfect home for you! 🏡 This custom-built mountain retreat in the Mountainside
neighborhood offers 4 bedrooms, 3 bathrooms, and stunning views. With a spacious house size of 2800
sqft, this home is listed below your budget at $400,000. Don't miss out on this dreamy mountain view
property! 🌄 #HomeSweetHome


### Below code is not needed for project just to check the version used and print it

In [17]:
import importlib
from importlib.metadata import version, PackageNotFoundError

PACKAGE_MAP = {
    "openai": "openai",
    "pandas": "pandas",
    "pydantic": "pydantic",
    "langchain": "langchain",
    "langchain_openai": "langchain-openai",
    "langchain_huggingface": "langchain-huggingface",
    "langchain_core": "langchain-core",
    "langchain_text_splitters": "langchain-text-splitters",
    "langchain_chroma": "langchain-chroma",
    "chromadb": "chromadb",
    "sentence_transformers": "sentence-transformers"
}

def get_version(import_name, pip_name):
    try:
        importlib.import_module(import_name)
    except ImportError:
        return "Not installed"

    try:
        return version(pip_name)
    except PackageNotFoundError:
        return "Version not found"

for import_name, pip_name in PACKAGE_MAP.items():
    print(f"{import_name}: {get_version(import_name, pip_name)}")


openai: 2.8.1
pandas: 2.3.3
pydantic: 2.12.4
langchain: 1.0.8
langchain_openai: 1.0.3
langchain_huggingface: 1.0.1
langchain_core: 1.0.7
langchain_text_splitters: 1.0.0
langchain_chroma: 1.0.0
chromadb: 1.3.5
sentence_transformers: 5.1.2
